# Introduction to Time Series Analysis

This notebook provides an interactive introduction to time series analysis concepts including:
- Basic time series visualization
- Heteroscedasticity (changing variance over time)
- Autocorrelation and Partial Autocorrelation Functions (ACF & PACF)
- **Stationarity Testing** with Augmented Dickey-Fuller (ADF) test
- **Detrending and Deseasonalizing** techniques
- Introduction to ARIMA models and forecasting

We'll use **Plotly** for interactive visualizations that allow you to explore the data dynamically.

🎯 **By the end of this notebook, you'll be able to:**
- Test whether a time series is stationary
- Transform non-stationary data into stationary data
- Read and interpret ACF/PACF plots
- Choose appropriate ARIMA parameters
- Build forecasting models


## 1. Setup and Imports

First, let's import the necessary libraries for our time series analysis.


In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy import signal, stats
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)


## 2. Basic Time Series Visualization

Let's start by creating and visualizing a simple time series. A time series is simply data points collected at successive time intervals.


In [2]:
# Generate a time series with trend and seasonality
n = 365  # one year of daily data
time = np.arange(n)
trend = 0.05 * time
seasonality = 10 * np.sin(2 * np.pi * time / 365)
noise = np.random.normal(0, 2, n)
ts_basic = trend + seasonality + noise + 50

# Create a date range
dates = pd.date_range(start='2023-01-01', periods=n, freq='D')

# Create interactive plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=dates, y=ts_basic, mode='lines', name='Time Series',
                         line=dict(color='blue', width=2)))

fig.update_layout(
    title='Basic Time Series: Trend + Seasonality + Noise',
    xaxis_title='Date',
    yaxis_title='Value',
    hovermode='x unified',
    template='plotly_white',
    height=500
)

fig.show()


## 3. Heteroscedasticity

**Heteroscedasticity** refers to the phenomenon where the variance of the error terms (or the series itself) changes over time. This is important to detect because many statistical models assume constant variance (homoscedasticity).

Let's visualize the difference between:
- **Homoscedastic** time series (constant variance)
- **Heteroscedastic** time series (increasing variance over time)


In [3]:
# Generate homoscedastic time series (constant variance)
n = 300
time = np.arange(n)
# Constant variance throughout - tight band around the mean
homoscedastic = 100 + 0.2 * time + np.random.normal(0, 3, n)

# Generate heteroscedastic time series (DRAMATICALLY increasing variance)
# Start with small variance, end with HUGE variance
variance_multiplier = (1 + time / 50) ** 2  # Quadratic growth - much more dramatic!
heteroscedastic = 100 + 0.2 * time + np.random.normal(0, 1, n) * variance_multiplier

# Create subplot with shared x-axis but independent y-axes
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('📊 Homoscedastic: Constant Variance (Notice the consistent spread)', 
                   '📈 Heteroscedastic: Exploding Variance (See how it fans out!)'),
    vertical_spacing=0.15
)

# Homoscedastic plot - nice tight consistent band
fig.add_trace(
    go.Scatter(x=time, y=homoscedastic, mode='lines', name='Homoscedastic',
               line=dict(color='green', width=1.5),
               fill='tonexty', fillcolor='rgba(0,255,0,0.1)'),
    row=1, col=1
)

# Heteroscedastic plot - starts tight, becomes wild!
fig.add_trace(
    go.Scatter(x=time, y=heteroscedastic, mode='lines', name='Heteroscedastic',
               line=dict(color='red', width=1.5),
               fill='tonexty', fillcolor='rgba(255,0,0,0.1)'),
    row=2, col=1
)

# Add trend lines to show the mean is same for both
trend_line = 100 + 0.2 * time
fig.add_trace(
    go.Scatter(x=time, y=trend_line, mode='lines', name='Mean Trend',
               line=dict(color='darkgreen', width=2, dash='dash'),
               showlegend=False),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=time, y=trend_line, mode='lines', name='Mean Trend',
               line=dict(color='darkred', width=2, dash='dash'),
               showlegend=False),
    row=2, col=1
)

fig.update_xaxes(title_text="Time", row=2, col=1)
fig.update_yaxes(title_text="Value", row=1, col=1)
fig.update_yaxes(title_text="Value", row=2, col=1)

fig.update_layout(
    title_text='🔍 Homoscedasticity vs Heteroscedasticity: Can You Spot the Difference?',
    showlegend=False,
    height=700,
    template='plotly_white'
)

fig.show()

print("👆 Notice:")
print("• TOP (Green): Spreads stays THE SAME throughout time")
print("• BOTTOM (Red): Spread gets WIDER and WIDER over time - like a funnel!")


👆 Notice:
• TOP (Green): Spreads stays THE SAME throughout time
• BOTTOM (Red): Spread gets WIDER and WIDER over time - like a funnel!


### Visualizing Variance Over Time

We can better understand heteroscedasticity by calculating and plotting the rolling variance:


In [4]:
# Calculate rolling variance with window size of 30
window = 30
df_homo = pd.DataFrame({'value': homoscedastic})
df_hetero = pd.DataFrame({'value': heteroscedastic})

rolling_var_homo = df_homo['value'].rolling(window=window).var()
rolling_var_hetero = df_hetero['value'].rolling(window=window).var()

# Create comparison plot
fig = go.Figure()

# Homoscedastic variance (should be flat/constant)
fig.add_trace(go.Scatter(
    x=time, y=rolling_var_homo,
    mode='lines', name='Homoscedastic (stays flat)',
    line=dict(color='green', width=3),
    fill='tozeroy', fillcolor='rgba(0,255,0,0.2)'
))

# Heteroscedastic variance (should EXPLODE upward!)
fig.add_trace(go.Scatter(
    x=time, y=rolling_var_hetero,
    mode='lines', name='Heteroscedastic (EXPLODES!)',
    line=dict(color='red', width=3),
    fill='tozeroy', fillcolor='rgba(255,0,0,0.2)'
))

# Add annotations
fig.add_annotation(
    x=100, y=rolling_var_homo.mean(),
    text="← Stays roughly constant",
    showarrow=True, arrowhead=2,
    arrowcolor='green', font=dict(color='green', size=12)
)

fig.add_annotation(
    x=250, y=rolling_var_hetero.values[250] if len(rolling_var_hetero) > 250 else rolling_var_hetero.max(),
    text="Skyrockets! 🚀",
    showarrow=True, arrowhead=2,
    arrowcolor='red', font=dict(color='red', size=12)
)

fig.update_layout(
    title=f'📊 Rolling Variance Over Time (window={window}) - Now You Can REALLY See It!',
    xaxis_title='Time',
    yaxis_title='Variance',
    hovermode='x unified',
    template='plotly_white',
    height=500,
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig.show()

print("\n💡 Key Insight:")
print(f"• Green line variance: ~{rolling_var_homo.mean():.1f} (constant)")
print(f"• Red line variance at end: ~{rolling_var_hetero.values[-1]:.1f} (HUGE!)")
print(f"• That's a {rolling_var_hetero.values[-1] / rolling_var_homo.mean():.0f}x difference!")



💡 Key Insight:
• Green line variance: ~12.6 (constant)
• Red line variance at end: ~1447.7 (HUGE!)
• That's a 115x difference!


## 4. Autocorrelation Function (ACF)

**Autocorrelation** measures the correlation between a time series and a lagged version of itself. It helps us understand:
- Whether past values influence future values
- The presence of patterns or cycles
- How many lags are significant

The ACF plot shows correlation coefficients for different lag values. Values outside the shaded confidence bands are considered statistically significant.


In [5]:
# Generate a time series with strong autocorrelation
n = 200
ar_series = np.zeros(n)
ar_series[0] = np.random.normal(0, 1)

# AR(1) process: y_t = 0.8 * y_{t-1} + noise
for t in range(1, n):
    ar_series[t] = 0.8 * ar_series[t-1] + np.random.normal(0, 1)

# Calculate ACF
acf_values = acf(ar_series, nlags=40, alpha=0.05)
acf_corr = acf_values[0]
acf_confint = acf_values[1]

# Create interactive ACF plot
lags = np.arange(len(acf_corr))
fig = go.Figure()

# Add stems (vertical lines)
for i in range(len(lags)):
    fig.add_trace(go.Scatter(
        x=[lags[i], lags[i]], 
        y=[0, acf_corr[i]],
        mode='lines',
        line=dict(color='blue', width=2),
        showlegend=False,
        hovertemplate=f'Lag: {lags[i]}<br>ACF: {acf_corr[i]:.3f}<extra></extra>'
    ))

# Add markers at the top of stems
fig.add_trace(go.Scatter(
    x=lags, y=acf_corr,
    mode='markers',
    marker=dict(size=8, color='blue'),
    name='ACF',
    hovertemplate='Lag: %{x}<br>ACF: %{y:.3f}<extra></extra>'
))

# Add confidence intervals
confidence_band = 1.96 / np.sqrt(len(ar_series))
fig.add_hline(y=confidence_band, line_dash="dash", line_color="gray", 
              annotation_text="95% Confidence", annotation_position="right")
fig.add_hline(y=-confidence_band, line_dash="dash", line_color="gray")
fig.add_hline(y=0, line_color="black", line_width=1)

fig.update_layout(
    title='Autocorrelation Function (ACF)',
    xaxis_title='Lag',
    yaxis_title='Correlation',
    template='plotly_white',
    height=500,
    showlegend=False
)

fig.show()


## 5. Partial Autocorrelation Function (PACF)

**Partial Autocorrelation** measures the correlation between a time series and its lag, after removing the effects of intermediate lags. This helps us:
- Identify the direct relationship between observations
- Determine the order of AR (AutoRegressive) models
- Distinguish direct effects from indirect effects

**Key Difference:**
- **ACF** shows both direct and indirect correlations
- **PACF** shows only direct correlations (controlling for intermediate lags)


In [6]:
# Calculate PACF
pacf_values = pacf(ar_series, nlags=40, alpha=0.05)
pacf_corr = pacf_values[0]
pacf_confint = pacf_values[1]

# Create interactive PACF plot
lags = np.arange(len(pacf_corr))
fig = go.Figure()

# Add stems (vertical lines)
for i in range(len(lags)):
    fig.add_trace(go.Scatter(
        x=[lags[i], lags[i]], 
        y=[0, pacf_corr[i]],
        mode='lines',
        line=dict(color='red', width=2),
        showlegend=False,
        hovertemplate=f'Lag: {lags[i]}<br>PACF: {pacf_corr[i]:.3f}<extra></extra>'
    ))

# Add markers at the top of stems
fig.add_trace(go.Scatter(
    x=lags, y=pacf_corr,
    mode='markers',
    marker=dict(size=8, color='red'),
    name='PACF',
    hovertemplate='Lag: %{x}<br>PACF: %{y:.3f}<extra></extra>'
))

# Add confidence intervals
confidence_band = 1.96 / np.sqrt(len(ar_series))
fig.add_hline(y=confidence_band, line_dash="dash", line_color="gray", 
              annotation_text="95% Confidence", annotation_position="right")
fig.add_hline(y=-confidence_band, line_dash="dash", line_color="gray")
fig.add_hline(y=0, line_color="black", line_width=1)

fig.update_layout(
    title='Partial Autocorrelation Function (PACF)',
    xaxis_title='Lag',
    yaxis_title='Partial Correlation',
    template='plotly_white',
    height=500,
    showlegend=False
)

fig.show()


### ACF vs PACF Side-by-Side Comparison

Let's compare ACF and PACF for the same series to see the difference:


In [7]:
# Create side-by-side comparison
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Autocorrelation Function (ACF)', 'Partial Autocorrelation Function (PACF)')
)

# ACF plot
for i in range(len(lags)):
    fig.add_trace(go.Scatter(
        x=[lags[i], lags[i]], y=[0, acf_corr[i]],
        mode='lines', line=dict(color='blue', width=2),
        showlegend=False
    ), row=1, col=1)

fig.add_trace(go.Scatter(
    x=lags, y=acf_corr, mode='markers',
    marker=dict(size=8, color='blue'),
    showlegend=False
), row=1, col=1)

# PACF plot
for i in range(len(lags)):
    fig.add_trace(go.Scatter(
        x=[lags[i], lags[i]], y=[0, pacf_corr[i]],
        mode='lines', line=dict(color='red', width=2),
        showlegend=False
    ), row=1, col=2)

fig.add_trace(go.Scatter(
    x=lags, y=pacf_corr, mode='markers',
    marker=dict(size=8, color='red'),
    showlegend=False
), row=1, col=2)

# Add confidence bands
fig.add_hline(y=confidence_band, line_dash="dash", line_color="gray", row=1, col=1)
fig.add_hline(y=-confidence_band, line_dash="dash", line_color="gray", row=1, col=1)
fig.add_hline(y=0, line_color="black", line_width=1, row=1, col=1)

fig.add_hline(y=confidence_band, line_dash="dash", line_color="gray", row=1, col=2)
fig.add_hline(y=-confidence_band, line_dash="dash", line_color="gray", row=1, col=2)
fig.add_hline(y=0, line_color="black", line_width=1, row=1, col=2)

fig.update_xaxes(title_text="Lag", row=1, col=1)
fig.update_xaxes(title_text="Lag", row=1, col=2)
fig.update_yaxes(title_text="Correlation", row=1, col=1)
fig.update_yaxes(title_text="Partial Correlation", row=1, col=2)

fig.update_layout(
    title_text='ACF vs PACF Comparison',
    height=500,
    template='plotly_white'
)

fig.show()


## 📖 How to Read ACF and PACF Plots: A Practical Guide

Understanding ACF and PACF plots is **crucial** for time series analysis and choosing ARIMA parameters. Let's break down how to interpret them!

### 🔍 Reading ACF Plots

**What are you looking at?**
- **X-axis (Lag)**: How many time steps back we're comparing
- **Y-axis (Correlation)**: Correlation coefficient (-1 to +1)
- **Blue bars**: Correlation value at each lag
- **Gray dashed lines**: 95% confidence interval (significance threshold)

**Key Rules:**
1. **Bars OUTSIDE the gray lines** = Statistically significant correlation
2. **Bars INSIDE the gray lines** = Not significant (likely just noise)
3. **Lag 0** is always 1.0 (perfect correlation with itself)

### 🔍 Reading PACF Plots

Same structure as ACF, but shows **direct** correlation only (removing indirect effects through intermediate lags).

### 🎯 Using ACF & PACF to Identify Time Series Patterns

| Pattern | ACF | PACF | Model Type |
|---------|-----|------|------------|
| **AR(p)** | Decays gradually (may oscillate/wiggle) | **Cuts off sharply** ✂️ after lag p | AutoRegressive |
| **MA(q)** | **Cuts off sharply** ✂️ after lag q | Decays gradually (may oscillate/wiggle) | Moving Average |
| **ARMA(p,q)** | Decays gradually (may oscillate/wiggle) | Decays gradually (may oscillate/wiggle) | Both AR and MA |

**Note**: "Decays gradually" can mean either smooth exponential decay OR oscillating/sinusoidal decay - both are decay as long as values stay within confidence bands!

### 📊 What to Look For:

1. **"Cuts off sharply"** = Spikes are significant, then suddenly drop to within confidence bands
   - Example: Lags 1, 2 are significant (bold) → Lag 3+ are not significant (gray)
   
2. **"Decays gradually"** = Values remain significant for many lags OR oscillate but stay insignificant
   - Can be **exponential decay**: 0.8 → 0.6 → 0.4 → 0.2 (smooth decrease)
   - Can be **sinusoidal decay**: 0.7 → -0.3 → 0.2 → -0.1 (wiggling/oscillating)
   - **Key point**: If values wiggle but stay INSIDE confidence bands = still decay!
   
3. **Count significant spikes** before cutoff to determine p and q values

⚠️ **Important**: "Decay" doesn't mean monotonic decrease! It can oscillate in a sine wave pattern. What matters is whether values are **statistically significant** (outside confidence bands) or not.


### 🧪 Visual Examples: Different Time Series Patterns

Let's generate three different types of series and see how their ACF/PACF plots differ!


In [8]:
# Generate three different processes
np.random.seed(42)
n = 300

# 1. AR(2) Process: y_t = 0.7*y_{t-1} - 0.3*y_{t-2} + noise
ar2_series = np.zeros(n)
for t in range(2, n):
    ar2_series[t] = 0.7 * ar2_series[t-1] - 0.3 * ar2_series[t-2] + np.random.normal(0, 1)

# 2. MA(2) Process: y_t = noise + 0.7*noise_{t-1} + 0.3*noise_{t-2}
noise = np.random.normal(0, 1, n)
ma2_series = np.zeros(n)
for t in range(2, n):
    ma2_series[t] = noise[t] + 0.7 * noise[t-1] + 0.3 * noise[t-2]

# 3. ARMA(1,1) Process: combination of AR(1) and MA(1)
arma_series = np.zeros(n)
noise_arma = np.random.normal(0, 1, n)
for t in range(1, n):
    arma_series[t] = 0.6 * arma_series[t-1] + noise_arma[t] + 0.4 * noise_arma[t-1]

# Calculate ACF and PACF for all three
series_dict = {
    'AR(2)': ar2_series,
    'MA(2)': ma2_series,
    'ARMA(1,1)': arma_series
}

# Create 3x2 subplot (3 rows, 2 columns: ACF and PACF for each)
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=('① AR(2) - ACF: DECAYS 📉', '① AR(2) - PACF: CUTS OFF ✂️ after lag 2',
                   '② MA(2) - ACF: CUTS OFF ✂️ after lag 2', '② MA(2) - PACF: DECAYS 📉',
                   '③ ARMA(1,1) - ACF: DECAYS 📉', '③ ARMA(1,1) - PACF: DECAYS 📉'),
    vertical_spacing=0.12,
    horizontal_spacing=0.12
)

row = 1
process_names = ['AR(2)', 'MA(2)', 'ARMA(1,1)']

for idx, (name, series) in enumerate(series_dict.items()):
    # Calculate ACF and PACF
    acf_vals = acf(series, nlags=15)  # Reduced lags for clarity
    pacf_vals = pacf(series, nlags=15)
    lags = np.arange(len(acf_vals))
    conf_band = 1.96 / np.sqrt(len(series))
    
    # ACF plot (column 1) - with color coding
    for i in range(len(lags)):
        # Color code: significant = bold blue, non-significant = light gray
        is_significant = abs(acf_vals[i]) > conf_band
        bar_color = 'rgb(0, 100, 255)' if is_significant else 'rgba(150, 150, 150, 0.4)'
        bar_width = 3 if is_significant else 2
        
        fig.add_trace(go.Scatter(
            x=[lags[i], lags[i]], y=[0, acf_vals[i]],
            mode='lines', line=dict(color=bar_color, width=bar_width),
            showlegend=False, 
            hovertemplate=f'Lag {lags[i]}<br>ACF: {acf_vals[i]:.3f}<br>{"✅ Significant" if is_significant else "❌ Not Significant"}<extra></extra>'
        ), row=row, col=1)
    
    fig.add_trace(go.Scatter(
        x=lags, y=acf_vals, mode='markers',
        marker=dict(size=8, color='blue'), showlegend=False
    ), row=row, col=1)
    
    # PACF plot (column 2) - with color coding
    for i in range(len(lags)):
        # Color code: significant = bold red, non-significant = light gray
        is_significant = abs(pacf_vals[i]) > conf_band
        bar_color = 'rgb(255, 50, 50)' if is_significant else 'rgba(150, 150, 150, 0.4)'
        bar_width = 3 if is_significant else 2
        
        fig.add_trace(go.Scatter(
            x=[lags[i], lags[i]], y=[0, pacf_vals[i]],
            mode='lines', line=dict(color=bar_color, width=bar_width),
            showlegend=False,
            hovertemplate=f'Lag {lags[i]}<br>PACF: {pacf_vals[i]:.3f}<br>{"✅ Significant" if is_significant else "❌ Not Significant"}<extra></extra>'
        ), row=row, col=2)
    
    fig.add_trace(go.Scatter(
        x=lags, y=pacf_vals, mode='markers',
        marker=dict(size=8, color='red'), showlegend=False
    ), row=row, col=2)
    
    # Add annotations to highlight patterns
    if name == 'AR(2)':
        # ACF decays
        fig.add_annotation(x=10, y=max(acf_vals[3:]) * 0.8, text="← Slowly decreasing",
                          showarrow=True, arrowhead=2, arrowcolor='blue',
                          font=dict(size=11, color='blue'), row=row, col=1)
        # PACF cuts off
        fig.add_annotation(x=3, y=pacf_vals[1] * 1.2, text="✂️ CUTS OFF",
                          showarrow=False, font=dict(size=12, color='red', family='Arial Black'),
                          bgcolor='rgba(255,255,0,0.3)', row=row, col=2)
        # Shade non-significant region in PACF
        fig.add_vrect(x0=2.5, x1=15, fillcolor="green", opacity=0.1,
                     layer="below", line_width=0, row=row, col=2)
        
    elif name == 'MA(2)':
        # ACF cuts off
        fig.add_annotation(x=3, y=acf_vals[1] * 1.2, text="✂️ CUTS OFF",
                          showarrow=False, font=dict(size=12, color='blue', family='Arial Black'),
                          bgcolor='rgba(255,255,0,0.3)', row=row, col=1)
        # Shade non-significant region in ACF
        fig.add_vrect(x0=2.5, x1=15, fillcolor="green", opacity=0.1,
                     layer="below", line_width=0, row=row, col=1)
        # PACF decays (with note about oscillation)
        fig.add_annotation(x=10, y=max(abs(pacf_vals[3:])) * 1.2, text="Oscillates but stays GRAY",
                          showarrow=True, arrowhead=2, arrowcolor='red',
                          font=dict(size=11, color='red'), row=row, col=2)
        fig.add_annotation(x=7, y=-max(abs(pacf_vals[3:])) * 0.9, text="(Sinusoidal decay is still decay!)",
                          showarrow=False, font=dict(size=9, color='darkred', style='italic'),
                          row=row, col=2)
        
    elif name == 'ARMA(1,1)':
        # Both decay
        fig.add_annotation(x=10, y=max(acf_vals[2:]) * 0.8, text="← Decays",
                          showarrow=True, arrowhead=2, arrowcolor='blue',
                          font=dict(size=11, color='blue'), row=row, col=1)
        fig.add_annotation(x=10, y=max(abs(pacf_vals[2:])) * 0.8, text="← Decays",
                          showarrow=True, arrowhead=2, arrowcolor='red',
                          font=dict(size=11, color='red'), row=row, col=2)
    
    # Add confidence bands and zero line
    for col in [1, 2]:
        # Confidence bands with shading
        fig.add_shape(type="rect", x0=0, x1=15, y0=-conf_band, y1=conf_band,
                     fillcolor="lightgray", opacity=0.2, layer="below", line_width=0,
                     row=row, col=col)
        fig.add_hline(y=conf_band, line_dash="dash", line_color="gray", 
                     line_width=2, row=row, col=col)
        fig.add_hline(y=-conf_band, line_dash="dash", line_color="gray", 
                     line_width=2, row=row, col=col)
        fig.add_hline(y=0, line_color="black", line_width=1, row=row, col=col)
    
    row += 1

# Update axes
for i in range(1, 4):
    fig.update_xaxes(title_text="Lag", row=i, col=1, range=[-0.5, 15])
    fig.update_xaxes(title_text="Lag", row=i, col=2, range=[-0.5, 15])
    fig.update_yaxes(title_text="ACF", row=i, col=1)
    fig.update_yaxes(title_text="PACF", row=i, col=2)

fig.update_layout(
    title_text='🎓 Learn to Read ACF & PACF: Visual Guide (Bold = Significant, Gray = Not Significant)',
    height=1000,
    template='plotly_white',
    showlegend=False
)

fig.show()

print("\n" + "="*80)
print("📚 INTERPRETATION GUIDE - How to Read These Plots")
print("="*80)

print("\n🔵 BLUE BARS = ACF (Autocorrelation)")
print("🔴 RED BARS = PACF (Partial Autocorrelation)")
print("✅ BOLD/DARK = Statistically Significant (outside gray bands)")
print("❌ LIGHT GRAY = Not Significant (inside gray bands)")
print("✂️ = Where it 'CUTS OFF'")
print("📉 = 'DECAYS GRADUALLY'")

print("\n" + "="*80)
print("1️⃣  AR(2) Process: y_t = 0.7*y_{t-1} - 0.3*y_{t-2} + noise")
print("="*80)
print("   🔵 ACF (LEFT): DECAYS GRADUALLY 📉")
print("      → Many lags remain significant (bold blue), slowly getting smaller")
print("      → May show smooth decay OR sinusoidal/oscillating decay")
print("   🔴 PACF (RIGHT): CUTS OFF ✂️ after lag 2")
print("      → Only lags 1 & 2 are significant (bold red)")
print("      → After lag 2, everything is gray = not significant")
print("   📝 CONCLUSION: Use p=2 for ARIMA (because PACF cuts off at 2)")

print("\n" + "="*80)
print("2️⃣  MA(2) Process: y_t = noise + 0.7*noise_{t-1} + 0.3*noise_{t-2}")
print("="*80)
print("   🔵 ACF (LEFT): CUTS OFF ✂️ after lag 2")
print("      → Only lags 1 & 2 are significant (bold blue)")
print("      → After lag 2, everything is gray = not significant")
print("   🔴 PACF (RIGHT): DECAYS GRADUALLY 📉 (with OSCILLATION)")
print("      → Notice the values wiggle up and down in a sine wave!")
print("      → BUT all those wiggles are GRAY (not significant)")
print("      → This sinusoidal/oscillating pattern is STILL decay!")
print("      → What matters: Values stay INSIDE confidence bands")
print("   📝 CONCLUSION: Use q=2 for ARIMA (because ACF cuts off at 2)")

print("\n" + "="*80)
print("3️⃣  ARMA(1,1) Process: Combination of AR and MA")
print("="*80)
print("   🔵 ACF (LEFT): DECAYS GRADUALLY 📉")
print("      → Multiple significant lags that slowly decrease")
print("   🔴 PACF (RIGHT): DECAYS GRADUALLY 📉")
print("      → Multiple significant lags that slowly decrease")
print("   📝 CONCLUSION: Need BOTH p and q (mixed model)")
print("      → Try ARIMA(1,1,1) or ARIMA(1,1,2) etc.")

print("\n" + "="*80)
print("💡 KEY TAKEAWAY:")
print("="*80)
print("Look for where the plot transitions from BOLD → GRAY")
print("That's where it 'cuts off'!")
print("\nIf you see: BOLD → GRAY at lag k → That's your p or q value!")
print("If you see: BOLD staying BOLD for many lags → That's 'decay'")
print("="*80)

print("\n" + "="*80)
print("❓ COMMON QUESTION: What does 'DECAY' mean?")
print("="*80)
print("\n'Decay' doesn't mean the values must decrease smoothly!")
print("\nDecay can happen in TWO ways:")
print("   1️⃣  EXPONENTIAL DECAY: Values get progressively smaller")
print("       Example: 0.8 → 0.6 → 0.4 → 0.2 → 0.1...")
print("\n   2️⃣  SINUSOIDAL DECAY (Dampened Oscillation): Values wiggle up/down")
print("       Example: 0.7 → -0.3 → 0.2 → -0.1 → 0.05...")
print("       ⚠️  Notice: It oscillates BUT stays within confidence bands!")
print("\n🔑 The KEY is: After several lags, the values stay INSIDE the gray bands")
print("   (not statistically significant anymore)")
print("\n👀 Look at MA(2) PACF: It wiggles/oscillates in a sine wave pattern,")
print("   but ALL those wiggles are GRAY (not significant) = DECAY!")
print("\n✂️ 'CUTS OFF' means: Significant → Not significant SUDDENLY")
print("📉 'DECAYS' means: Values stay not significant (even if wiggling)")
print("="*80)



📚 INTERPRETATION GUIDE - How to Read These Plots

🔵 BLUE BARS = ACF (Autocorrelation)
🔴 RED BARS = PACF (Partial Autocorrelation)
✅ BOLD/DARK = Statistically Significant (outside gray bands)
❌ LIGHT GRAY = Not Significant (inside gray bands)
✂️ = Where it 'CUTS OFF'
📉 = 'DECAYS GRADUALLY'

1️⃣  AR(2) Process: y_t = 0.7*y_{t-1} - 0.3*y_{t-2} + noise
   🔵 ACF (LEFT): DECAYS GRADUALLY 📉
      → Many lags remain significant (bold blue), slowly getting smaller
      → May show smooth decay OR sinusoidal/oscillating decay
   🔴 PACF (RIGHT): CUTS OFF ✂️ after lag 2
      → Only lags 1 & 2 are significant (bold red)
      → After lag 2, everything is gray = not significant
   📝 CONCLUSION: Use p=2 for ARIMA (because PACF cuts off at 2)

2️⃣  MA(2) Process: y_t = noise + 0.7*noise_{t-1} + 0.3*noise_{t-2}
   🔵 ACF (LEFT): CUTS OFF ✂️ after lag 2
      → Only lags 1 & 2 are significant (bold blue)
      → After lag 2, everything is gray = not significant
   🔴 PACF (RIGHT): DECAYS GRADUALLY 📉 (wi

### 📝 Quick Reference Cheat Sheet

**Step-by-Step: How to Determine ARIMA(p, d, q) Parameters**

```
1. Check for Stationarity (determines 'd')
   - Plot your data. Does it have a trend or changing variance?
   - If YES → need differencing (d=1 or d=2)
   - If NO → already stationary (d=0)

2. Look at PACF (determines 'p')
   - Count how many lags are OUTSIDE confidence bands before it cuts off
   - That number = p (AR order)
   - Example: PACF significant at lags 1, 2, then drops → p=2

3. Look at ACF (determines 'q')
   - Count how many lags are OUTSIDE confidence bands before it cuts off
   - That number = q (MA order)
   - Example: ACF significant at lags 1, 2, 3, then drops → q=3

4. Start Simple, Then Refine
   - Begin with ARIMA(1,1,1) if unsure
   - Check residuals - should look like white noise
   - Adjust p, d, q based on model diagnostics
```

**Common Patterns Quick Guide:**

| If you see... | Then use... |
|--------------|-------------|
| PACF cuts off at lag 2, ACF decays | AR(2) → ARIMA(2, d, 0) |
| ACF cuts off at lag 1, PACF decays | MA(1) → ARIMA(0, d, 1) |
| Both decay gradually | ARMA → ARIMA(p, d, q) - try both |
| Nothing significant | White noise (no model needed!) |

**Pro Tips:**
- 🎯 Most real-world series: d=1 (one difference)
- 🎯 Keep p and q small (typically 0-3)
- 🎯 Simpler is better (principle of parsimony)


## 5.5. Testing for Stationarity: The Foundation of Time Series Modeling

Before we can use ARIMA or other classical time series models, we need to ensure our data is **stationary**. This section will teach you how to test for stationarity and transform non-stationary data.

### What is Stationarity?

A time series is **stationary** when its statistical properties remain constant over time:
- ✅ **Constant Mean**: No upward or downward trend
- ✅ **Constant Variance**: Spread stays the same (no heteroscedasticity!)
- ✅ **Constant Autocorrelation**: Correlation structure doesn't change

### Why Does It Matter?

- Most time series models (ARIMA, ARMA, etc.) **assume stationarity**
- Non-stationary data can lead to spurious correlations and unreliable forecasts
- We need to transform non-stationary data → stationary data before modeling


### 📊 The Augmented Dickey-Fuller (ADF) Test

The **ADF test** is the most common statistical test for stationarity. 

**Hypothesis:**
- **Null Hypothesis (H₀)**: The series has a unit root (i.e., it's NON-stationary)
- **Alternative Hypothesis (H₁)**: The series is stationary

**How to Interpret:**
- **p-value < 0.05**: Reject H₀ → Series IS stationary ✅
- **p-value ≥ 0.05**: Fail to reject H₀ → Series is NOT stationary ❌

Let's test this on real data!


In [9]:
from statsmodels.tsa.stattools import adfuller

# Function to perform and display ADF test results
def adf_test(series, name=''):
    """Perform Augmented Dickey-Fuller test and display results"""
    result = adfuller(series, autolag='AIC')
    
    print(f'\n{"="*60}')
    print(f'ADF Test Results for: {name}')
    print(f'{"="*60}')
    print(f'ADF Statistic: {result[0]:.6f}')
    print(f'p-value: {result[1]:.6f}')
    print(f'Critical Values:')
    for key, value in result[4].items():
        print(f'   {key}: {value:.3f}')
    
    # Interpretation
    if result[1] <= 0.05:
        print(f"\n✅ Result: STATIONARY (p-value = {result[1]:.6f} < 0.05)")
        print("   → Can proceed with modeling!")
    else:
        print(f"\n❌ Result: NON-STATIONARY (p-value = {result[1]:.6f} ≥ 0.05)")
        print("   → Need to transform the data (detrend/deseasonalize/difference)")
    
    return result

# Generate two time series: one stationary, one non-stationary
np.random.seed(42)
n = 300

# 1. Stationary series (white noise + small AR component)
stationary_series = np.random.normal(0, 1, n)
for i in range(1, n):
    stationary_series[i] += 0.3 * stationary_series[i-1]

# 2. Non-stationary series (random walk with drift + trend)
non_stationary = np.zeros(n)
for i in range(1, n):
    non_stationary[i] = non_stationary[i-1] + 0.5 + np.random.normal(0, 1)

# Visualize both
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Stationary Series', 'Non-Stationary Series (Random Walk with Drift)'),
    vertical_spacing=0.15
)

fig.add_trace(
    go.Scatter(x=np.arange(n), y=stationary_series, mode='lines',
               line=dict(color='green', width=1.5), name='Stationary'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(n), y=non_stationary, mode='lines',
               line=dict(color='red', width=1.5), name='Non-Stationary'),
    row=2, col=1
)

fig.update_xaxes(title_text="Time", row=2, col=1)
fig.update_yaxes(title_text="Value", row=1, col=1)
fig.update_yaxes(title_text="Value", row=2, col=1)

fig.update_layout(
    title_text='Visual Comparison: Stationary vs Non-Stationary',
    height=600,
    showlegend=False,
    template='plotly_white'
)

fig.show()

# Perform ADF test on both
adf_test(stationary_series, 'Stationary Series')
adf_test(non_stationary, 'Non-Stationary Series')



ADF Test Results for: Stationary Series
ADF Statistic: -13.774184
p-value: 0.000000
Critical Values:
   1%: -3.452
   5%: -2.871
   10%: -2.572

✅ Result: STATIONARY (p-value = 0.000000 < 0.05)
   → Can proceed with modeling!

ADF Test Results for: Non-Stationary Series
ADF Statistic: -1.739642
p-value: 0.410812
Critical Values:
   1%: -3.452
   5%: -2.871
   10%: -2.572

❌ Result: NON-STATIONARY (p-value = 0.410812 ≥ 0.05)
   → Need to transform the data (detrend/deseasonalize/difference)


(np.float64(-1.7396421591839393),
 np.float64(0.41081200082898456),
 0,
 299,
 {'1%': np.float64(-3.4524113009049935),
  '5%': np.float64(-2.8712554127251764),
  '10%': np.float64(-2.571946570731871)},
 np.float64(789.5352730524393))

### 🔧 Making Data Stationary: Transformation Techniques

When data is non-stationary, we have several techniques to transform it:

1. **Detrending**: Remove the trend component
2. **Deseasonalizing**: Remove seasonal patterns
3. **Differencing**: Subtract previous values
4. **Log Transform**: Stabilize variance (for heteroscedasticity)

Let's explore each method with interactive examples!


In [10]:
# ====================
# 1. DETRENDING
# ====================

from scipy import signal

# Create a series with a strong trend
np.random.seed(123)
n = 365
time = np.arange(n)

# Original: Trend + Noise
trend = 0.2 * time  # Linear trend
noise = np.random.normal(0, 5, n)
series_with_trend = 100 + trend + noise

# METHOD 1: Linear Detrending (using scipy)
detrended_linear = signal.detrend(series_with_trend, type='linear')

# METHOD 2: Polynomial Detrending (fit and subtract polynomial)
poly_coeffs = np.polyfit(time, series_with_trend, deg=2)  # 2nd degree polynomial
poly_trend = np.polyval(poly_coeffs, time)
detrended_poly = series_with_trend - poly_trend

# METHOD 3: Differencing (simple and common)
differenced = np.diff(series_with_trend)

# Create 4-panel visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Original Series (Non-Stationary)', 
                   'Linear Detrending',
                   'Polynomial Detrending',
                   'First Differencing'),
    vertical_spacing=0.15,
    horizontal_spacing=0.12
)

# Original
fig.add_trace(
    go.Scatter(x=time, y=series_with_trend, mode='lines',
               line=dict(color='red', width=1.5), name='Original'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=time, y=trend + 100, mode='lines',
               line=dict(color='darkred', width=2, dash='dash'), name='Trend',
               showlegend=False),
    row=1, col=1
)

# Linear Detrending
fig.add_trace(
    go.Scatter(x=time, y=detrended_linear, mode='lines',
               line=dict(color='green', width=1.5), name='Detrended'),
    row=1, col=2
)
fig.add_hline(y=0, line_dash="dash", line_color="black", row=1, col=2)

# Polynomial Detrending
fig.add_trace(
    go.Scatter(x=time, y=detrended_poly, mode='lines',
               line=dict(color='blue', width=1.5), name='Detrended'),
    row=2, col=1
)
fig.add_hline(y=0, line_dash="dash", line_color="black", row=2, col=1)

# Differencing
fig.add_trace(
    go.Scatter(x=time[:-1], y=differenced, mode='lines',
               line=dict(color='purple', width=1.5), name='Differenced'),
    row=2, col=2
)
fig.add_hline(y=0, line_dash="dash", line_color="black", row=2, col=2)

# Update axes
for i in range(1, 3):
    for j in range(1, 3):
        fig.update_xaxes(title_text="Time", row=i, col=j)
        fig.update_yaxes(title_text="Value", row=i, col=j)

fig.update_layout(
    title_text='🔧 Detrending Methods Comparison',
    height=700,
    showlegend=False,
    template='plotly_white'
)

fig.show()

# Test stationarity on each
print("🧪 ADF Test Results:")
adf_test(series_with_trend, 'Original (with trend)')
adf_test(detrended_linear, 'Linear Detrending')
adf_test(detrended_poly, 'Polynomial Detrending')
adf_test(differenced, 'First Differencing')


🧪 ADF Test Results:

ADF Test Results for: Original (with trend)
ADF Statistic: -0.182031
p-value: 0.940630
Critical Values:
   1%: -3.449
   5%: -2.870
   10%: -2.571

❌ Result: NON-STATIONARY (p-value = 0.940630 ≥ 0.05)
   → Need to transform the data (detrend/deseasonalize/difference)

ADF Test Results for: Linear Detrending
ADF Statistic: -18.737146
p-value: 0.000000
Critical Values:
   1%: -3.448
   5%: -2.870
   10%: -2.571

✅ Result: STATIONARY (p-value = 0.000000 < 0.05)
   → Can proceed with modeling!

ADF Test Results for: Polynomial Detrending
ADF Statistic: -18.740825
p-value: 0.000000
Critical Values:
   1%: -3.448
   5%: -2.870
   10%: -2.571

✅ Result: STATIONARY (p-value = 0.000000 < 0.05)
   → Can proceed with modeling!

ADF Test Results for: First Differencing
ADF Statistic: -7.001748
p-value: 0.000000
Critical Values:
   1%: -3.449
   5%: -2.870
   10%: -2.571

✅ Result: STATIONARY (p-value = 0.000000 < 0.05)
   → Can proceed with modeling!


(np.float64(-7.001747987183061),
 np.float64(7.291836072795732e-10),
 16,
 347,
 {'1%': np.float64(-3.449336554273722),
  '5%': np.float64(-2.8699055166063085),
  '10%': np.float64(-2.571226758215748)},
 np.float64(2126.6770541389287))

In [11]:
# ====================
# 2. DESEASONALIZING
# ====================

from statsmodels.tsa.seasonal import seasonal_decompose

# Create a series with strong seasonality
np.random.seed(456)
n = 365 * 2  # 2 years of daily data
time = np.arange(n)

# Components
trend = 0.1 * time
seasonality = 20 * np.sin(2 * np.pi * time / 365)  # Annual seasonality
noise = np.random.normal(0, 3, n)
series_with_seasonality = 100 + trend + seasonality + noise

# METHOD 1: Using statsmodels seasonal_decompose
decomposition = seasonal_decompose(series_with_seasonality, model='additive', period=365)

# Extract components
observed = decomposition.observed
trend_component = decomposition.trend
seasonal_component = decomposition.seasonal
residual_component = decomposition.resid

# Deseasonalized = Original - Seasonal component
deseasonalized = series_with_seasonality - seasonal_component

# Create comprehensive visualization
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=('Original Series (with Seasonality)', 
                   'Deseasonalized Series',
                   'Trend Component',
                   'Seasonal Component',
                   'Residual (Noise)',
                   'Deseasonalized vs Original'),
    vertical_spacing=0.10,
    horizontal_spacing=0.12
)

# Original
fig.add_trace(
    go.Scatter(x=time, y=series_with_seasonality, mode='lines',
               line=dict(color='red', width=1), name='Original'),
    row=1, col=1
)

# Deseasonalized
fig.add_trace(
    go.Scatter(x=time, y=deseasonalized, mode='lines',
               line=dict(color='green', width=1), name='Deseasonalized'),
    row=1, col=2
)

# Trend
fig.add_trace(
    go.Scatter(x=time, y=trend_component, mode='lines',
               line=dict(color='blue', width=2), name='Trend'),
    row=2, col=1
)

# Seasonal
fig.add_trace(
    go.Scatter(x=time[:365], y=seasonal_component[:365], mode='lines',
               line=dict(color='orange', width=2), name='Seasonal'),
    row=2, col=2
)

# Residual
fig.add_trace(
    go.Scatter(x=time, y=residual_component, mode='lines',
               line=dict(color='purple', width=0.5), name='Residual'),
    row=3, col=1
)
fig.add_hline(y=0, line_dash="dash", line_color="black", row=3, col=1)

# Comparison
fig.add_trace(
    go.Scatter(x=time, y=series_with_seasonality, mode='lines',
               line=dict(color='red', width=1), name='Original',
               opacity=0.5),
    row=3, col=2
)
fig.add_trace(
    go.Scatter(x=time, y=deseasonalized, mode='lines',
               line=dict(color='green', width=1.5), name='Deseasonalized'),
    row=3, col=2
)

# Update axes
for i in range(1, 4):
    for j in range(1, 3):
        fig.update_xaxes(title_text="Time", row=i, col=j)
        fig.update_yaxes(title_text="Value", row=i, col=j)

fig.update_layout(
    title_text='🌊 Seasonal Decomposition: Breaking Down the Components',
    height=900,
    showlegend=False,
    template='plotly_white'
)

fig.show()

# Test stationarity
print("\n🧪 ADF Test Results:")
adf_test(series_with_seasonality, 'Original (with seasonality)')
adf_test(deseasonalized, 'Deseasonalized')

# Also test on residuals (fully decomposed)
# Remove NaN values from residuals first
residual_clean = residual_component[~np.isnan(residual_component)]
adf_test(residual_clean, 'Residuals (Trend + Seasonality removed)')



🧪 ADF Test Results:

ADF Test Results for: Original (with seasonality)
ADF Statistic: -0.455801
p-value: 0.900357
Critical Values:
   1%: -3.440
   5%: -2.866
   10%: -2.569

❌ Result: NON-STATIONARY (p-value = 0.900357 ≥ 0.05)
   → Need to transform the data (detrend/deseasonalize/difference)

ADF Test Results for: Deseasonalized
ADF Statistic: 0.885491
p-value: 0.992898
Critical Values:
   1%: -3.440
   5%: -2.866
   10%: -2.569

❌ Result: NON-STATIONARY (p-value = 0.992898 ≥ 0.05)
   → Need to transform the data (detrend/deseasonalize/difference)

ADF Test Results for: Residuals (Trend + Seasonality removed)
ADF Statistic: 0.421888
p-value: 0.982295
Critical Values:
   1%: -3.449
   5%: -2.870
   10%: -2.571

❌ Result: NON-STATIONARY (p-value = 0.982295 ≥ 0.05)
   → Need to transform the data (detrend/deseasonalize/difference)


(np.float64(0.42188779693513123),
 np.float64(0.98229526883437),
 16,
 349,
 {'1%': np.float64(-3.449226932880019),
  '5%': np.float64(-2.869857365438656),
  '10%': np.float64(-2.571201085130664)},
 np.float64(-561.3704591916155))

### 📝 Summary: Choosing the Right Transformation

Here's a quick decision tree for making your data stationary:

```
START: Run ADF Test on your data
   |
   ├─ p-value < 0.05? ✅ STATIONARY → Proceed with modeling!
   |
   └─ p-value ≥ 0.05? ❌ NON-STATIONARY → Transform it:
       |
       ├─ Has TREND?
       |   └─ Try: Detrending or Differencing
       |
       ├─ Has SEASONALITY?
       |   └─ Try: Seasonal Decomposition (remove seasonal component)
       |
       ├─ Has BOTH?
       |   └─ Try: 1) Deseasonalize first, 2) Then detrend/difference
       |
       └─ Has CHANGING VARIANCE (Heteroscedasticity)?
           └─ Try: Log transform, then detrend/deseasonalize
```

**Common Approaches:**

| Problem | Solution | When to Use |
|---------|----------|-------------|
| **Linear Trend** | First Differencing | Most common, simple |
| **Polynomial Trend** | Polynomial Detrending | Curved trends |
| **Seasonality** | `seasonal_decompose` | Regular seasonal patterns |
| **Trend + Seasonality** | Decompose → Detrend | Both present |
| **Changing Variance** | Log transform first | Heteroscedastic data |
| **Unit Root** | Differencing (d=1 or d=2) | Random walk behavior |

**Pro Tips:**
- 🎯 **Always test with ADF after transformation** to verify stationarity
- 🎯 **Start simple**: Try differencing first (it's used in ARIMA's "I" component)
- 🎯 **Don't over-difference**: If d=1 makes it stationary, don't use d=2
- 🎯 **Visual inspection helps**: Plot your data before and after transformation


### 🧑‍💻 Practice: Complete Workflow Example

Let's put it all together with a realistic time series that has trend, seasonality, and noise!


In [12]:
# ====================
# COMPLETE WORKFLOW: Simulating Sales Data
# ====================

# Generate realistic "sales" data with multiple components
np.random.seed(789)
n = 365 * 3  # 3 years of daily data
time = np.arange(n)
dates = pd.date_range(start='2021-01-01', periods=n, freq='D')

# Components of sales data
trend = 0.05 * time  # Growing business
seasonality = 30 * np.sin(2 * np.pi * time / 365)  # Annual pattern
weekly = 10 * np.sin(2 * np.pi * time / 7)  # Weekly pattern (weekday/weekend)
noise = np.random.normal(0, 8, n)
sales_data = 200 + trend + seasonality + weekly + noise

# Create DataFrame
df = pd.DataFrame({'date': dates, 'sales': sales_data})
df.set_index('date', inplace=True)

print("📊 STEP-BY-STEP WORKFLOW FOR MAKING DATA STATIONARY")
print("="*70)

# STEP 1: Visualize original data
print("\n1️⃣  STEP 1: Visualize & Inspect Original Data")
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['sales'], mode='lines',
                         line=dict(color='red', width=1),
                         name='Original Sales Data'))
fig.update_layout(title='Original Sales Data (Non-Stationary)',
                  xaxis_title='Date', yaxis_title='Sales',
                  template='plotly_white', height=400)
fig.show()

# STEP 2: Test for stationarity
print("\n2️⃣  STEP 2: Test for Stationarity (ADF Test)")
adf_test(df['sales'], 'Original Sales Data')

# STEP 3: Decompose to understand components
print("\n3️⃣  STEP 3: Decompose to Identify Components")
decomp = seasonal_decompose(df['sales'], model='additive', period=365)

# STEP 4: Remove seasonality
print("\n4️⃣  STEP 4: Remove Seasonality")
df['deseasonalized'] = df['sales'] - decomp.seasonal
adf_test(df['deseasonalized'], 'After Deseasonalizing')

# STEP 5: Apply differencing to remove trend
print("\n5️⃣  STEP 5: Apply First Differencing")
df['differenced'] = df['deseasonalized'].diff()
df_clean = df.dropna()  # Remove NaN from differencing
adf_test(df_clean['differenced'], 'After Deseasonalizing + Differencing')

# STEP 6: Visualize the transformation journey
print("\n6️⃣  STEP 6: Visualize the Transformation Journey")
fig = make_subplots(
    rows=4, cols=1,
    subplot_titles=('① Original (Non-Stationary)', 
                   '② After Deseasonalizing',
                   '③ After Differencing',
                   '④ Final Stationary Series (Ready for ARIMA!)'),
    vertical_spacing=0.08
)

fig.add_trace(
    go.Scatter(x=df.index, y=df['sales'], mode='lines',
               line=dict(color='red', width=1)),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=df.index, y=df['deseasonalized'], mode='lines',
               line=dict(color='orange', width=1)),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=df_clean.index, y=df_clean['differenced'], mode='lines',
               line=dict(color='green', width=1)),
    row=3, col=1
)

# Highlight final stationary series
fig.add_trace(
    go.Scatter(x=df_clean.index, y=df_clean['differenced'], mode='lines',
               line=dict(color='darkgreen', width=1.5)),
    row=4, col=1
)
fig.add_hline(y=0, line_dash="dash", line_color="black", row=4, col=1)

for i in range(1, 5):
    fig.update_xaxes(title_text="Date", row=i, col=1)
    fig.update_yaxes(title_text="Value", row=i, col=1)

fig.update_layout(
    title_text='🎯 Complete Transformation: Non-Stationary → Stationary',
    height=1000,
    showlegend=False,
    template='plotly_white'
)

fig.show()

print("\n" + "="*70)
print("✅ SUCCESS! The data is now stationary and ready for ARIMA modeling!")
print("="*70)
print("\n💡 Key Takeaways:")
print("   • Original data: Non-stationary (p > 0.05)")
print("   • After deseasonalizing: Still non-stationary (trend remains)")
print("   • After differencing: STATIONARY (p < 0.05) ✅")
print("   • We would use: d=1 in ARIMA (one difference)")
print("\n📈 Next step: Fit ARIMA model to the stationary series!")


📊 STEP-BY-STEP WORKFLOW FOR MAKING DATA STATIONARY

1️⃣  STEP 1: Visualize & Inspect Original Data



2️⃣  STEP 2: Test for Stationarity (ADF Test)

ADF Test Results for: Original Sales Data
ADF Statistic: -1.369093
p-value: 0.597000
Critical Values:
   1%: -3.436
   5%: -2.864
   10%: -2.568

❌ Result: NON-STATIONARY (p-value = 0.597000 ≥ 0.05)
   → Need to transform the data (detrend/deseasonalize/difference)

3️⃣  STEP 3: Decompose to Identify Components

4️⃣  STEP 4: Remove Seasonality

ADF Test Results for: After Deseasonalizing
ADF Statistic: -0.775758
p-value: 0.826139
Critical Values:
   1%: -3.436
   5%: -2.864
   10%: -2.568

❌ Result: NON-STATIONARY (p-value = 0.826139 ≥ 0.05)
   → Need to transform the data (detrend/deseasonalize/difference)

5️⃣  STEP 5: Apply First Differencing

ADF Test Results for: After Deseasonalizing + Differencing
ADF Statistic: -13.641557
p-value: 0.000000
Critical Values:
   1%: -3.436
   5%: -2.864
   10%: -2.568

✅ Result: STATIONARY (p-value = 0.000000 < 0.05)
   → Can proceed with modeling!

6️⃣  STEP 6: Visualize the Transformation Journey



✅ SUCCESS! The data is now stationary and ready for ARIMA modeling!

💡 Key Takeaways:
   • Original data: Non-stationary (p > 0.05)
   • After deseasonalizing: Still non-stationary (trend remains)
   • After differencing: STATIONARY (p < 0.05) ✅
   • We would use: d=1 in ARIMA (one difference)

📈 Next step: Fit ARIMA model to the stationary series!


## 6. Introduction to ARIMA Models

**ARIMA** stands for **AutoRegressive Integrated Moving Average**. It's one of the most popular time series forecasting methods.

### ARIMA Components: ARIMA(p, d, q)

1. **AR (AutoRegressive) - p**: Uses past values to predict future values
   - Model: y_t = c + φ₁y_{t-1} + φ₂y_{t-2} + ... + φₚy_{t-p} + ε_t
   - **p** = number of lag observations

2. **I (Integrated) - d**: Number of times the data needs to be differenced to make it stationary
   - **d** = order of differencing (usually 0, 1, or 2)
   - Stationary data has constant mean and variance over time

3. **MA (Moving Average) - q**: Uses past forecast errors to predict future values
   - Model: y_t = μ + ε_t + θ₁ε_{t-1} + θ₂ε_{t-2} + ... + θqε_{t-q}
   - **q** = number of lagged forecast errors

### How to Choose p, d, q:
- **p**: Look at PACF plot - where it cuts off
- **d**: Number of differencing needed for stationarity (often 1)
- **q**: Look at ACF plot - where it cuts off


### ARIMA Example: Forecasting

Let's create a time series, fit an ARIMA model, and make forecasts:


In [13]:
# Generate a more complex time series
np.random.seed(123)
n = 200
time_idx = np.arange(n)

# Create trend + seasonality + noise
trend = 0.5 * time_idx
seasonality = 15 * np.sin(2 * np.pi * time_idx / 50)
noise = np.random.normal(0, 3, n)
ts_data = 100 + trend + seasonality + noise

# Split into train and test
train_size = 150
train = ts_data[:train_size]
test = ts_data[train_size:]

# Fit ARIMA model
# Based on our ACF/PACF analysis, we'll use ARIMA(2, 1, 2)
model = ARIMA(train, order=(2, 1, 2))
fitted_model = model.fit()

# Make predictions
n_forecast = len(test)
forecast = fitted_model.forecast(steps=n_forecast)

# Get confidence intervals
forecast_result = fitted_model.get_forecast(steps=n_forecast)
forecast_ci = forecast_result.conf_int()

print(fitted_model.summary())


                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  150
Model:                 ARIMA(2, 1, 2)   Log Likelihood                -421.527
Date:                Mon, 20 Oct 2025   AIC                            853.053
Time:                        12:53:21   BIC                            868.073
Sample:                             0   HQIC                           859.155
                                - 150                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          1.2713      0.110     11.569      0.000       1.056       1.487
ar.L2         -0.3357      0.112     -2.987      0.003      -0.556      -0.115
ma.L1         -1.8239      0.051    -35.431      0.0

In [14]:
# Create visualization of ARIMA forecast
fig = go.Figure()

# Training data
fig.add_trace(go.Scatter(
    x=time_idx[:train_size],
    y=train,
    mode='lines',
    name='Training Data',
    line=dict(color='blue', width=2)
))

# Test data (actual)
fig.add_trace(go.Scatter(
    x=time_idx[train_size:],
    y=test,
    mode='lines',
    name='Actual Test Data',
    line=dict(color='green', width=2)
))

# Forecast
fig.add_trace(go.Scatter(
    x=time_idx[train_size:],
    y=forecast,
    mode='lines',
    name='ARIMA Forecast',
    line=dict(color='red', width=2, dash='dash')
))

# Confidence intervals
# Convert to numpy array if it's a DataFrame, otherwise use as is
ci_array = forecast_ci.values if hasattr(forecast_ci, 'values') else forecast_ci

fig.add_trace(go.Scatter(
    x=time_idx[train_size:],
    y=ci_array[:, 0],  # Lower bound
    mode='lines',
    name='95% CI Lower',
    line=dict(width=0),
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=time_idx[train_size:],
    y=ci_array[:, 1],  # Upper bound
    mode='lines',
    name='95% CI Upper',
    line=dict(width=0),
    fillcolor='rgba(255, 0, 0, 0.2)',
    fill='tonexty',
    showlegend=True
))

# Add vertical line to separate train/test
fig.add_vline(x=train_size, line_dash="dash", line_color="gray", 
              annotation_text="Train/Test Split", annotation_position="top")

fig.update_layout(
    title='ARIMA(2,1,2) Forecast with 95% Confidence Interval',
    xaxis_title='Time',
    yaxis_title='Value',
    hovermode='x unified',
    template='plotly_white',
    height=500,
    legend=dict(x=0.02, y=0.98)
)

fig.show()


### ARIMA Model Diagnostics

Let's examine the residuals (errors) from our ARIMA model to ensure it's a good fit:


In [15]:
# Get residuals
residuals = fitted_model.resid

# Create diagnostic plots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Residuals Over Time', 'Residuals Histogram',
                   'Residuals ACF', 'Q-Q Plot'),
    specs=[[{"type": "scatter"}, {"type": "histogram"}],
           [{"type": "scatter"}, {"type": "scatter"}]]
)

# 1. Residuals over time
fig.add_trace(
    go.Scatter(x=np.arange(len(residuals)), y=residuals,
               mode='lines', line=dict(color='purple', width=1)),
    row=1, col=1
)
fig.add_hline(y=0, line_dash="dash", line_color="red", row=1, col=1)

# 2. Histogram of residuals
fig.add_trace(
    go.Histogram(x=residuals, nbinsx=30, marker_color='lightblue',
                 name='Residuals'),
    row=1, col=2
)

# 3. ACF of residuals
residuals_acf = acf(residuals, nlags=30)
lags_resid = np.arange(len(residuals_acf))

for i in range(len(lags_resid)):
    fig.add_trace(go.Scatter(
        x=[lags_resid[i], lags_resid[i]], 
        y=[0, residuals_acf[i]],
        mode='lines',
        line=dict(color='blue', width=2),
        showlegend=False
    ), row=2, col=1)

fig.add_trace(
    go.Scatter(x=lags_resid, y=residuals_acf, mode='markers',
               marker=dict(size=6, color='blue'), showlegend=False),
    row=2, col=1
)

confidence_band_resid = 1.96 / np.sqrt(len(residuals))
fig.add_hline(y=confidence_band_resid, line_dash="dash", line_color="gray", row=2, col=1)
fig.add_hline(y=-confidence_band_resid, line_dash="dash", line_color="gray", row=2, col=1)
fig.add_hline(y=0, line_color="black", row=2, col=1)

# 4. Q-Q plot (theoretical quantiles vs sample quantiles)
from scipy import stats
theoretical_quantiles = stats.norm.ppf(np.linspace(0.01, 0.99, len(residuals)))
sample_quantiles = np.sort(residuals)

fig.add_trace(
    go.Scatter(x=theoretical_quantiles, y=sample_quantiles,
               mode='markers', marker=dict(size=5, color='orange'),
               name='Q-Q Plot'),
    row=2, col=2
)

# Add reference line for Q-Q plot
fig.add_trace(
    go.Scatter(x=theoretical_quantiles, y=theoretical_quantiles,
               mode='lines', line=dict(color='red', dash='dash'),
               name='Reference', showlegend=False),
    row=2, col=2
)

fig.update_xaxes(title_text="Time", row=1, col=1)
fig.update_xaxes(title_text="Residual Value", row=1, col=2)
fig.update_xaxes(title_text="Lag", row=2, col=1)
fig.update_xaxes(title_text="Theoretical Quantiles", row=2, col=2)

fig.update_yaxes(title_text="Residual", row=1, col=1)
fig.update_yaxes(title_text="Frequency", row=1, col=2)
fig.update_yaxes(title_text="ACF", row=2, col=1)
fig.update_yaxes(title_text="Sample Quantiles", row=2, col=2)

fig.update_layout(
    title_text='ARIMA Model Diagnostics',
    height=700,
    showlegend=False,
    template='plotly_white'
)

fig.show()


## 7. Interactive Exploration: Try Different Parameters!

Now it's your turn! Try modifying the ARIMA parameters below to see how they affect the forecast:


In [16]:
# EXPERIMENT HERE: Try changing these parameters!
p_value = 1  # AR order (try 0, 1, 2, 3)
d_value = 1  # Differencing order (try 0, 1, 2)
q_value = 1  # MA order (try 0, 1, 2, 3)

print(f"Fitting ARIMA({p_value}, {d_value}, {q_value})...")

try:
    # Fit model with your parameters
    model_custom = ARIMA(train, order=(p_value, d_value, q_value))
    fitted_custom = model_custom.fit()
    
    # Make forecast
    forecast_custom = fitted_custom.forecast(steps=n_forecast)
    
    # Calculate error metrics
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    mae = mean_absolute_error(test, forecast_custom)
    rmse = np.sqrt(mean_squared_error(test, forecast_custom))
    
    print(f"\n✓ Model fitted successfully!")
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
    
    # Visualize
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=time_idx[:train_size], y=train,
        mode='lines', name='Training Data',
        line=dict(color='blue', width=2)
    ))
    
    fig.add_trace(go.Scatter(
        x=time_idx[train_size:], y=test,
        mode='lines', name='Actual Test Data',
        line=dict(color='green', width=2)
    ))
    
    fig.add_trace(go.Scatter(
        x=time_idx[train_size:], y=forecast_custom,
        mode='lines', name=f'ARIMA({p_value},{d_value},{q_value}) Forecast',
        line=dict(color='red', width=2, dash='dash')
    ))
    
    fig.add_vline(x=train_size, line_dash="dash", line_color="gray")
    
    fig.update_layout(
        title=f'Your ARIMA({p_value},{d_value},{q_value}) Model - MAE: {mae:.2f}, RMSE: {rmse:.2f}',
        xaxis_title='Time',
        yaxis_title='Value',
        hovermode='x unified',
        template='plotly_white',
        height=500
    )
    
    fig.show()
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("Try different parameters!")


Fitting ARIMA(1, 1, 1)...

✓ Model fitted successfully!
Mean Absolute Error (MAE): 14.79
Root Mean Squared Error (RMSE): 16.54


## 8. Key Takeaways & Summary

### What We Learned:

1. **Time Series Components**:
   - Trend: Long-term direction
   - Seasonality: Regular patterns
   - Noise: Random variation

2. **Heteroscedasticity**:
   - When variance changes over time
   - Important to detect for model assumptions
   - Can be visualized with rolling variance

3. **Autocorrelation (ACF)**:
   - Measures correlation with lagged values
   - Shows both direct and indirect relationships
   - Helps identify MA component (q)

4. **Partial Autocorrelation (PACF)**:
   - Measures correlation after removing intermediate effects
   - Shows only direct relationships
   - Helps identify AR component (p)

5. **ARIMA Models**:
   - **AR(p)**: Uses past values
   - **I(d)**: Differencing for stationarity
   - **MA(q)**: Uses past errors
   - Choose parameters using ACF/PACF plots

### Practice Exercises:

1. **Exercise 1**: Modify the heteroscedasticity example to create a series with decreasing variance
2. **Exercise 2**: Try different ARIMA parameters and compare their MAE/RMSE scores
3. **Exercise 3**: Create your own time series with different seasonal patterns
4. **Exercise 4**: Analyze the ACF and PACF plots to determine appropriate (p,q) values

### Further Learning:
- SARIMA: ARIMA with seasonal components
- VAR: Vector AutoRegression for multivariate series
- Prophet: Facebook's forecasting tool
- LSTM: Deep learning for time series
